In [1]:
!pip install transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 217.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 180.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 236.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 195.3 MB/s eta 0:00:0000:01


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

In [10]:
import torch
import base64

In [11]:
checkpoint = "MBZUAI/LaMini-T5-738M"

In [12]:
## Loading the tokenizer and model from the HuggingFace to Local

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [13]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 157.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 137.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 195.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.6/580.6 kB 46.0 MB/s eta 0:00:00
  Attempting uninstall: marshmallow
    Found existing installation: marshmallow 4.0.0
    Uninstalling marshmallow-4.0.0:
      Successfully uninstalled marshmallow-4.0.0


In [14]:
!pip install langchain-huggingface

In [15]:
from langchain_huggingface import HuggingFacePipeline
def slm_pipeline():
    pipe = pipeline(
        "text2text-generation",
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p = 0.95
    )

    local_slm = HuggingFacePipeline(pipeline = pipe)
    return local_slm

In [16]:
input_prompt = "Write an article about WEB3 and its usecases"

In [17]:
model = slm_pipeline()

Device set to use cpu


In [18]:
gen_text = model.invoke(input_prompt)

In [19]:
gen_text

'WEB3 stands for World Wide Web Consortium, which is a global web development platform that provides web-based services to developers around the world. It is a platform that allows developers to create web pages, videos, and other content that are accessible to the general public. WEB3 has a wide range of use cases, including: 1. Web development: WEB3 is a web development platform that provides web development services to developers. It is designed to be user-friendly and efficient. 2. Web development: WEB3 is a web development platform that provides web hosting services to developers. It is used to'

In [7]:
!pip show sagemaker

Name: sagemaker
Version: 2.243.3
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk
Author: Amazon Web Services
Author-email: 
License: 
Location: /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages
Requires: attrs, boto3, cloudpickle, docker, fastapi, google-pasta, importlib-metadata, jsonschema, numpy, omegaconf, packaging, pandas, pathos, platformdirs, protobuf, psutil, pyyaml, requests, sagemaker-core, schema, smdebug-rulesconfig, tblib, tqdm, urllib3, uvicorn
Required-by: 


### Deployment - Deploy model on Sagemaker

In [21]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
	'HF_TASK': 'text2texxt-generation',
    'device_map': 'auto',
    'torch_dtype': 'torch.float32'
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.2.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Write an article about Cyber Security",
})

[05/10/25 12:01:06] INFO     Found credentials from IAM Role:                                   ]8;id=555896;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=135804;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=818639;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=89492;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/10/25 12:01:07] INFO     Defaulting to only available Python version: py311                   ]8;id=860112;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=745956;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: gpu.                       ]8;id=661691;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=212447;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=771587;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=120582;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Creating model with name:                                              ]8;id=957849;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=320553;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-2025-05-10-12-01-07-722                             

[05/10/25 12:01:08] INFO     Creating endpoint-config with name                                     ]8;id=305693;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=732742;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             huggingface-pytorch-tgi-inference-2025-05-10-12-01-08-253                             

                    INFO     Creating endpoint with name                                            ]8;id=493381;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=254333;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\
                             huggingface-pytorch-tgi-inference-2025-05-10-12-01-08-253                             

-------------!

[{'generated_text': 'Cyber Security is the practice of protecting electronic systems and systems from unauthorized access, theft, and disruption. Cybercrime can occur either through malicious online activities or human error. Regardless of the reason, they can harm individuals and society as a whole. It is a major concern for organizations, governments, and individuals alike to keep their systems and systems safe from cyber attacks. One of the most understated aspects of cyber security is the use of secure computer systems. While traditional computer networks are not foolproof and can be vulnerable to hacking or other security vulnerabilities, they rely on strong passwords, accessibility and authentication'}]

### SageMaker Endpoints - Use deployed endpoints

In [27]:
ENDPOINT = "huggingface-pytorch-tgi-inference-2025-05-10-12-01-08-253"

In [28]:
import boto3

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

endpoint_name = ENDPOINT

In [33]:
## API Payload

prompt = "Write an article about Cyber Security and WEB3"

payload = {
    'inputs' : prompt,
    'parameters': {
        'max_new_tokens' : 256,
        'do_sample' : True,
        'temperature' : 0.3,
        'top_p' : 0.7,
        'top_k' : 50,
        'repetion_penalty' : 1.03
    }
}

response = sagemaker_runtime.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = "application/json",
    Body = json.dumps(payload)
)

predictions = json.loads(response['Body'].read().decode('utf-8'))
final_result =predictions[0]['generated_text']
print(final_result)

Cyber Security and WEB3 are two of the most important aspects of cyber security today. WEB3 stands for World Wide Web, which is a global web browser that provides secure and accessible web pages. It is designed to provide users with a secure and efficient way to access and share information online. One of the key benefits of WEB3 is that it helps to protect sensitive information from unauthorized access. This is particularly important for organizations that operate on the internet. By using WEB3, organizations can create a secure and accessible web environment that is accessible to users from anywhere in the world. This can include websites that
